In [24]:

from keras.models import Sequential
from keras.layers import Dense
from keras import regularizers
from sklearn import metrics
from keras.models import model_from_json
from sklearn.preprocessing import MinMaxScaler
from keras.layers import Dropout
from sklearn.model_selection import train_test_split
import numpy
import time
import pandas
from keras import optimizers
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import Normalizer

In [25]:

print('loading data')

dataframe = pandas.read_csv("train_newcsd_neutron.txt", delimiter=",", header=None)
dataset=dataframe.values
# split into input (X) and output (Y) variables
X = dataset[:,0:33]
Y = dataset[:,33]

print('splitting data')

X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.15, random_state=50)


loading data
splitting data


In [26]:
print('building network')



class_weight = {0: 1.0,
                1: 1.0,
               }
# used in unbalanced datasets 


neurons=256

def create_model(drop=0.0,l2val=0.0):  #drop and l2val are going to be tuned later 
    model = Sequential()
    model.add(Dense(neurons, input_dim=33, activation='relu',kernel_regularizer=regularizers.l2(l2val)))
    model.add(Dropout(drop))
    model.add(Dense(neurons, activation='relu',kernel_regularizer=regularizers.l2(l2val)))
    model.add(Dropout(drop))
    model.add(Dense(neurons, activation='relu',kernel_regularizer=regularizers.l2(l2val)))
    model.add(Dropout(drop))
    model.add(Dense(neurons, activation='relu',kernel_regularizer=regularizers.l2(l2val)))
    model.add(Dropout(drop))
    model.add(Dense(neurons, activation='relu',kernel_regularizer=regularizers.l2(l2val)))
    model.add(Dropout(drop))
    model.add(Dense(neurons, activation='relu',kernel_regularizer=regularizers.l2(l2val)))
    model.add(Dropout(drop))
    model.add(Dense(neurons, activation='relu',kernel_regularizer=regularizers.l2(l2val)))
    model.add(Dropout(drop))
    model.add(Dense(neurons, activation='relu',kernel_regularizer=regularizers.l2(l2val)))
    model.add(Dropout(drop))
    model.add(Dense(neurons, activation='relu',kernel_regularizer=regularizers.l2(l2val)))    
    model.add(Dropout(drop))
    model.add(Dense(1, activation='sigmoid'))

    epochs = 300;
    learning_rate = 0.001
    decay_rate = learning_rate / epochs * 10
    adam=optimizers.Adam(lr=learning_rate, decay=decay_rate, beta_1=0.9, beta_2=0.999, epsilon=0.001)
    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

    return model

building network


In [27]:
l2val=[0.0001,0.0005,0.00005]
drop=[0.1,0.3,0.5]

model = KerasClassifier(build_fn=create_model, epochs=epochs, batch_size=1024, verbose=0)


param_grid = dict(l2val=l2val,drop=drop)
grid = GridSearchCV(estimator=model, param_grid=param_grid)
#grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=5) ## the defualt cross validation is 3

print('finish building')

finish building


In [28]:
print('start tuning')
print('please do not disturb')

grid_result = grid.fit(X_train,Y_train)


print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']   #the arverage accuracy among the 3 folds 
stds = grid_result.cv_results_['std_test_score']   # the standard deviation. 
params = grid_result.cv_results_['params']   
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

    
    
# usually the highest mean with lowest std is the best
# higher accuray -> better performance 
# higher std -> more likely overtraining 

start tuning
please do not disturb
Best: 0.972910 using {'drop': 0.1, 'l2val': 0.0001}
0.972910 (0.000234) with: {'drop': 0.1, 'l2val': 0.0001}
0.972519 (0.000640) with: {'drop': 0.1, 'l2val': 0.0005}
0.972445 (0.000908) with: {'drop': 0.1, 'l2val': 5e-05}
0.966504 (0.002104) with: {'drop': 0.3, 'l2val': 0.0001}
0.969237 (0.000940) with: {'drop': 0.3, 'l2val': 0.0005}
0.968087 (0.001101) with: {'drop': 0.3, 'l2val': 5e-05}
0.851452 (0.010874) with: {'drop': 0.5, 'l2val': 0.0001}
0.883660 (0.010154) with: {'drop': 0.5, 'l2val': 0.0005}
0.839410 (0.013836) with: {'drop': 0.5, 'l2val': 5e-05}


In [ ]:
neurons=256
#lr is learning rate  # beta_1 is momentum, this one is not that important, you can only search for lr
drop=0.5
l2val=0.0005 #(use the values you just determined by last grid search )

def create_model(lr=0.0,beta_1=0.0):  
    model = Sequential()
    model.add(Dense(neurons, input_dim=33, activation='relu',kernel_regularizer=regularizers.l2(l2val)))
    model.add(Dropout(drop))
    model.add(Dense(neurons, activation='relu',kernel_regularizer=regularizers.l2(l2val)))
    model.add(Dropout(drop))
    model.add(Dense(neurons, activation='relu',kernel_regularizer=regularizers.l2(l2val)))
    model.add(Dropout(drop))
    model.add(Dense(neurons, activation='relu',kernel_regularizer=regularizers.l2(l2val)))
    model.add(Dropout(drop))
    model.add(Dense(neurons, activation='relu',kernel_regularizer=regularizers.l2(l2val)))
    model.add(Dropout(drop))

    model.add(Dense(neurons, activation='relu',kernel_regularizer=regularizers.l2(l2val)))
    model.add(Dropout(drop))
    model.add(Dense(neurons, activation='relu',kernel_regularizer=regularizers.l2(l2val)))
    model.add(Dropout(drop))
    model.add(Dense(1, activation='sigmoid'))


    adam=optimizers.Adam(lr=lr, beta_1=beta_1, beta_2=0.999, epsilon=0.001, decay=0.001)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
lr = [0.001, 0.0008, 0.0006, 0.0005, 0.0004]
beta_1 = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
model = KerasClassifier(build_fn=create_model, epochs=600, batch_size=2048, verbose=0)

param_grid = dict(lr=lr, beta_1=beta_1)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train, Y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
############  you can also tune the number of neurons in each layer 
############  but this is usually the same as tuning l2 and drop (they are both directly related to the complexity of the strucutre)
############  I think people usually fix the number of neurons and layers and reduce the complexity (reduce over-training) by tuning l2 and drop


drop=0.5
l2val=0.0005 
lr=???
beta_1 =???

def create_model(neurons=128):  
    model = Sequential()
    model.add(Dense(neurons, input_dim=184, activation='relu',kernel_regularizer=regularizers.l2(l2val)))
    model.add(Dropout(drop))
    model.add(Dense(neurons, activation='relu',kernel_regularizer=regularizers.l2(l2val)))
    model.add(Dropout(drop))
    model.add(Dense(neurons, activation='relu',kernel_regularizer=regularizers.l2(l2val)))
    model.add(Dropout(drop))
    model.add(Dense(neurons, activation='relu',kernel_regularizer=regularizers.l2(l2val)))
    model.add(Dropout(drop))
    model.add(Dense(neurons, activation='relu',kernel_regularizer=regularizers.l2(l2val)))
    model.add(Dropout(drop))

    model.add(Dense(neurons, activation='relu',kernel_regularizer=regularizers.l2(l2val)))
    model.add(Dropout(drop))
    model.add(Dense(neurons, activation='relu',kernel_regularizer=regularizers.l2(l2val)))
    model.add(Dropout(drop))
    model.add(Dense(1, activation='sigmoid'))

    
neurons = [64,128,256,512,1024]
model = KerasClassifier(build_fn=create_model, epochs=600, batch_size=1024, verbose=0)

param_grid = dict(neurons)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train, Y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
